# Ridge Regression Model for ESA CCI Water Vapor Data, predicted by ERA5
This notebook implements a ridge regression model to predict ESA CCI water vapor data using ERA5 reanalysis data as predictors. The model is trained on historical data and evaluated on a separate test set.
The first step is to load the corresponding datasets for pre-processing them into the format necessary for ridge regression. Also the necessary libraries will be imported.

In [1]:
## Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import netCDF4
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score
import sklearn
from sklearn.model_selection import KFold

In [8]:
## Import the datasets. Required datasets are the result from data-processor.ipynb as well as ERA5
fn_cci_processed = '../data/timeseries/esa_cci_timeseries.nc'
fn_era5 = "../data/era5/era5_1984_2020_5degree_zipped.nc"
data_cci = netCDF4.Dataset(fn_cci_processed)
data_era5 = netCDF4.Dataset(fn_era5)

values_cci = data_cci['value'][:,:]

# Look at the shape of data_cci
print(data_cci.variables.keys())
print(values_cci.shape)

## Define the indices for non nan values in the cci dataset
non_nan_indices = np.argwhere(~np.isnan(values_cci[0,:])).T
print(non_nan_indices[:])
print(non_nan_indices[:].shape)

dict_keys(['time', 'index', 'value'])
(420, 420)
[[ 48  50  51  52  54  56  57  60  72  74  75  76  99 100 102 104 105 106
  108 110 111 112 114 116 117 118 120 123 124 126 128 129 130 132 134 135
  136 137 138 139 140 141 143 144 145 146 147 149 151 152 153 155 156 157
  158 159 161 162 163 164 165 167 169 170 171 173 175 176 177 179 181 182
  183 185 187 188 189 191 193 194 195 197 199 200 201 203 205 206 207 209
  210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227
  228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245
  246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263
  264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281
  282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299
  300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317
  318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335
  336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 3

# Load and prepare ERA5 data for ridge regression
We want to have the ERA5 predictors as simple arrays in order to train the ridge regression on each predictor individually. Therefore we will extract the relevant variables from the netCDF file and reshape them accordingly.

In [26]:
nt_train_end=420
nt_train_start=0
# Extract the relevant variable from ERA5 dataset
# First show the relevant keys
print(data_era5.variables.keys())
# show latitudes and longitudes in the dataset ; these are directly the ones we want to use for the training later on, but we want the latitude to be reversed
print(data_era5['latitude'][:])
print(data_era5['longitude'][:])
era5_lat = data_era5['latitude'][:][::-1]
era5_lon = data_era5['longitude'][:]
era5_ta = data_era5['t'][nt_train_start:nt_train_end,:-1,::-1,:]
era5_levels = data_era5['pressure_level'][:-1]
era5_vs = data_era5['q'][nt_train_start:nt_train_end,:,::-1,:]
## Show the pressure levels available
print(era5_levels)
## Show the values available
print(era5_ta.shape)
print(era5_vs.shape)


dict_keys(['number', 'valid_time', 'pressure_level', 'latitude', 'longitude', 'expver', 'q', 't', 'u', 'v'])
[ 60.  55.  50.  45.  40.  35.  30.  25.  20.  15.  10.   5.   0.  -5.
 -10. -15. -20. -25. -30. -35. -40. -45. -50. -55. -60.]
[-180. -175. -170. -165. -160. -155. -150. -145. -140. -135. -130. -125.
 -120. -115. -110. -105. -100.  -95.  -90.  -85.  -80.  -75.  -70.  -65.
  -60.  -55.  -50.  -45.  -40.  -35.  -30.  -25.  -20.  -15.  -10.   -5.
    0.    5.   10.   15.   20.   25.   30.   35.   40.   45.   50.   55.
   60.   65.   70.   75.   80.   85.   90.   95.  100.  105.  110.  115.
  120.  125.  130.  135.  140.  145.  150.  155.  160.  165.  170.  175.]
[175. 150. 125. 100.  70.]
(420, 5, 25, 72)
(420, 6, 25, 72)


### Set Ridge Regression parameters
Now we will set the parameters for the ridge regression. This includes the alpha values to be tested during cross-validation as well as the number of splits for time series cross-validation.
We will train all ridge regressions individually in the following order: temperature (reproduction of Nowack et al. (2023)), vertical wind speed, u wind component, v wind component.

In [ ]:
# Set alpha values
alpha_i=[0.0001,0.0003,0.001,0.003,0.01,0.03,0.1,0.3,1,2,4,6,8,10,12,14,16,18,20,25,30,40,50,60,70,80,90,100,150,200,250,300,400,500,600,700,800,900,1000,3000,5000,10000,12000,14000,16000,18000,20000,25000,30000,50000,100000,300000,1e6,3e6,1e7,3e7,1e8]

In [35]:
nr_lat=len(era5_lat)
nr_lon=len(era5_lon)
nr_levels=len(era5_levels)
print(nr_lat,nr_lon,nr_levels)
lag=2

25 72 5


In [39]:
## Reformat the ERA5 data into 2D arrays for ridge regression

X_raw_hist_era5 = era5_ta.reshape(era5_ta.shape[0], nr_lon*nr_lat*(nr_levels))
print(X_raw_hist_era5.shape)
non_nan_indices = non_nan_indices.flatten()
non_nan_indices_lag = non_nan_indices-lag

(420, 9000)


#### Start the training for temperature predictors
We will now start the training of the ridge regression model using temperature as predictor. The model will be trained on the training set and evaluated on the test set.

In [43]:
X_lag_hist_era5 = np.hstack((X_raw_hist_era5[lag:,:],X_raw_hist_era5[lag-1:-1,:],X_raw_hist_era5[lag-2:-2,:]))
print(X_lag_hist_era5.shape)
n_cci = values_cci.shape[1]
## Select months with available data
X_lag_hist_era5_cci = X_lag_hist_era5[non_nan_indices_lag]
print(X_lag_hist_era5_cci.shape)
## Prepare the scaler
scaler_era5 = StandardScaler()
x_scaler_era5 = scaler_era5.fit(X_lag_hist_era5_cci[:,:])
x_train_norm_era5 = x_scaler_era5.transform(X_lag_hist_era5_cci[:,:])
era5_alphas = []
era5_predictions = np.empty((n_cci,X_lag_hist_era5_cci.shape[0]))

(418, 27000)
(300, 27000)
